In [4]:
import pandas as pd
import numpy as np

In [1]:
import psycopg2
conn=psycopg2.connect(database="ernie",user="wenxi",host="localhost",password="temp_ERNIE_1234")
conn.set_client_encoding('UTF8')
conn.autocommit=True
curs=conn.cursor()

### Theta_Plus Visualization

In [2]:
curs.execute("SET SEARCH_PATH TO theta_plus;")

In [7]:
node_size = pd.read_sql(
'''
WITH cte AS (SELECT a.cluster_no, a.scp, b.auid FROM theta_plus.imm1985_1995_cluster_scp_list_unshuffled a INNER JOIN public.scopus_authors b
ON a.scp = b.scp
WHERE cluster_no = 15865)
SELECT cte.auid, COUNT(*) as node_size FROM cte
GROUP BY cte.auid
''', conn)

In [11]:
node_size.head()

,auid,node_size
0,6503951189,1
1,6504069823,1
2,6506298135,1
3,6506562124,2
4,6506618842,1


In [9]:
edge_size = pd.read_sql(
'''
WITH cte AS(
    SELECT a.scp, b.auid FROM theta_plus.imm1985_1995_cluster_scp_list_unshuffled a
    INNER JOIN public.scopus_authors b
               ON a.scp = b.scp
     WHERE a.cluster_no = 15865)
SELECT author_1, author_2, COUNT(*) FROM (
SELECT cte.scp, cte.auid AS author_1, c.auid AS author_2 FROM cte JOIN cte c ON cte.scp = c.scp AND cte.auid != c.auid) d
GROUP BY author_1, author_2
''', conn)

In [12]:
edge_size.head()

,author_1,author_2,count
0,7101621010,57193203418,1
1,7006846682,7102706945,4
2,57194482830,7201440236,1
3,7102706945,57190035381,1
4,7102706945,35918707600,1


In [14]:
import networkx as nx
from netwulf import visualize

G = nx.from_pandas_edgelist(edge_size, 'author_1', 'author_2')

i=0
# Set link 'weight' attributes
for n1, n2, data in G.edges(data=True):
    data['weight'] = edge_size['count'][i] * 10
    i+=1

i=0
# Set node 'size' attributes
for n, data in G.nodes(data=True):
    data['size'] = node_size['node_size'][i]
    i+=1

visualize(G)

(None, None)